<a href="https://colab.research.google.com/github/LoganLxb/tensorflow2.0-learn/blob/main/tensorflow2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf


x=tf.Variable(initial_value=3.)

with tf.GradientTape() as tape:
  y= tf.square(x,2)
y_grad=tape.gradient(y,x)
print(y,y_grad)

tf.Tensor(9.0, shape=(), dtype=float32) tf.Tensor(6.0, shape=(), dtype=float32)


In [ ]:
X=tf.constant([[1.,2.],[3.,4.]])
y=tf.constant([[1.],[2.]])

w=tf.Variable([[1.],[2.]])
b=tf.Variable(initial_value=1.)

with tf.GradientTape() as tape1:
  L = tf.reduce_sum(tf.square(tf.matmul(X, w) + b - y))
w_grad,b_grad=tape1.gradient(L,[w,b])

print(w_grad,b_grad)

tf.Tensor(
[[ 70.]
 [100.]], shape=(2, 1), dtype=float32) tf.Tensor(30.0, shape=(), dtype=float32)


In [ ]:
import numpy as np
x_raw=np.array([2013,2014,2015,2016,2017],dtype=np.float32)
y_raw=np.array([12000, 14000, 15000, 16500, 17500],dtype=np.float32)\

X=(x_raw-x_raw.min())/(x_raw.max()-x_raw.min())
y=(y_raw-y_raw.min())/(y_raw.max()-y_raw.min())

x=tf.constant(X)
y=tf.constant(y)


a=tf.Variable(initial_value=0.)
b=tf.Variable(initial_value=0.)

variable=[a,b]
epoch=10000
optimzer=tf.keras.optimizers.SGD(learning_rate=5e-4)
for e in range(epoch):
  with tf.GradientTape() as tape:
    y_pred=a*x+b
    loss=tf.reduce_sum(tf.square(y_pred-y))
  grad=tape.gradient(loss,variable)
  optimzer.apply_gradients(grads_and_vars=zip(grad, variable))

print(a,b)

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=0.97637> <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=0.057565063>


In [ ]:
X = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
y = tf.constant([[10.0], [20.0]])


class Linear(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.dense = tf.keras.layers.Dense(
            units=1,#输出张量的维度；
            activation=None,#激活函数
            kernel_initializer=tf.zeros_initializer(),#初始化权重
            bias_initializer=tf.zeros_initializer()#初始化偏置
        )

    def call(self, input):
        output = self.dense(input)
        return output

epoch=100

model = Linear()
optimzer=tf.keras.optimizers.SGD(learning_rate=5e-4)#optimizer优化器  对权重和偏置做剃度下降
for e in range(epoch):
  with tf.GradientTape() as tape:
    y_pred=model(X)
    loss = tf.reduce_mean(tf.square(y_pred - y))
  grads = tape.gradient(loss, model.variables)    # 使用 model.variables 这一属性直接获得模型中的所有变量
  optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))
print(model.variables)

[<tf.Variable 'linear_3/dense_5/kernel:0' shape=(3, 1) dtype=float32, numpy=
array([[0.935125 ],
       [1.2844828],
       [1.6338407]], dtype=float32)>, <tf.Variable 'linear_3/dense_5/bias:0' shape=(1,) dtype=float32, numpy=array([0.3493583], dtype=float32)>]


In [ ]:
#cnn 手写数字
import tensorflow as tf
import numpy as np
class MNISTLoader():
    def __init__(self):
        mnist = tf.keras.datasets.mnist
        (self.train_data, self.train_label), (self.test_data, self.test_label) = mnist.load_data()
        # MNIST中的图像默认为uint8（0-255的数字）。以下代码将其归一化到0-1之间的浮点数，并在最后增加一维作为颜色通道
        self.train_data = np.expand_dims(self.train_data.astype(np.float32) / 255.0, axis=-1)      # [60000, 28, 28, 1]
        self.test_data = np.expand_dims(self.test_data.astype(np.float32) / 255.0, axis=-1)        # [10000, 28, 28, 1]
        self.train_label = self.train_label.astype(np.int32)    # [60000]
        self.test_label = self.test_label.astype(np.int32)      # [10000]
        self.num_train_data, self.num_test_data = self.train_data.shape[0], self.test_data.shape[0]

    def get_batch(self, batch_size):
        # 从数据集中随机取出batch_size个元素并返回
        index = np.random.randint(0, self.num_train_data, batch_size)
        return self.train_data[index, :], self.train_label[index]
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()    # Flatten层将除第一维（batch_size）以外的维度展平
        self.dense1 = tf.keras.layers.Dense(units=100, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(units=10)

    def call(self, inputs):         # [batch_size, 28, 28, 1]
        x = self.flatten(inputs)    # [batch_size, 784]
        x = self.dense1(x)          # [batch_size, 100]
        x = self.dense2(x)          # [batch_size, 10]
        output = tf.nn.softmax(x)
        return output
num_epochs = 5
batch_size = 50
learning_rate = 0.001
model = MLP()
data_loader = MNISTLoader()
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
num_batches = int(data_loader.num_train_data // batch_size * num_epochs)
for batch_index in range(num_batches):
    X, y = data_loader.get_batch(batch_size)
    with tf.GradientTape() as tape:
        y_pred = model(X)
        loss = tf.keras.losses.sparse_categorical_crossentropy(y_true=y, y_pred=y_pred)
        loss = tf.reduce_mean(loss)
        print("batch %d: loss %f" % (batch_index, loss.numpy()))
    grads = tape.gradient(loss, model.variables)
    optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))
loss = tf.keras.losses.sparse_categorical_crossentropy(y_true=y, y_pred=y_pred)
sparse_categorical_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
   

In [5]:
sparse_categorical_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
num_batches = int(data_loader.num_test_data // batch_size)
for batch_index in range(num_batches):
    start_index, end_index = batch_index * batch_size, (batch_index + 1) * batch_size
    y_pred = model.predict(data_loader.test_data[start_index: end_index])
    sparse_categorical_accuracy.update_state(y_true=data_loader.test_label[start_index: end_index], y_pred=y_pred)
print("test accuracy: %f" % sparse_categorical_accuracy.result())

test accuracy: 0.974300
